Etapas para automatização total (quase total) da sobrevivência utilizando imagens de vant

1. Plotagem de parcelas (VOCÊ ESTÁ AQUI!);
   
   1.1 Atualizar base de dados do Pós;
   
   1.2 Criar pasta e gdb para receber base temporária;
   
       1.2.1 Parcelas / talhões; 
       
   1.3 Codificar parcelas;
   
   1.4 Alimentar a base do GEO (10.0.2.227);

2. Contagem automática por imagem;
    
    2.1 Corte das imagens na área de parcelas;
    
    2.2 Segmentação e classificação;
    
    2.3 Identificação das mudas;
    
    2.4 Centroid das mudas vivas;
    
    2.4 Alimentar base de contagem;

3. Processo manual para identificação das falhas;
    
    3.1 Análise parcela a parcela no GIS;
    
        3.1.1 Plotar falhas;
        
        3.1.2 Alterar talhões;
        
    3.2 Alteração de status - gatilho para próximas etapas automáticas;
    
4. Indicadores;

 (Cenas para os próximos capítulos);

In [ ]:
import shutil
import arcpy
import sys
import os
import time
from datetime import datetime, timedelta

#Habilitar autocompletar
%config IPCompleter.greedy=True

#Liberar overwriteoutput
arcpy.env.overwriteOutput = True

#Data
Date = time.strftime("%Y%m%d")
days_ago = datetime.now() - timedelta(days=1)
days_ago_str = days_ago.strftime("%m/%d/%Y")

#Nome do projeto
projname = "Plot_parcelas_"+Date

In [ ]:
#Local
local = r'C:\Users\KLDB00132941\Documents\Klabin\IFS_VANT\DEV'

In [ ]:
# Criar pasta
folder = str(arcpy.CreateFolder_management(local, projname)) + "\\"

#Criar gdb
gdb = arcpy.CreateFileGDB_management(folder,projname)

In [ ]:
#Converter arquivo xlsx em table
table = arcpy.ExcelToTable_conversion(r'\\10.0.2.194\pr\07 - Inventario\03_IFS_VANT\05_automatizacao\Pos_plantio.xlsx' , gdb)

In [ ]:
#Renomear arquivo tipo table
arcpy.Rename_management(table,"pos_atualizado")

In [ ]:
#Importar table para dentro do gdb criado

dbf = folder+"\\"+"pos_atualizado.dbf"

table2 = arcpy.TableToGeodatabase_conversion (dbf , gdb)

In [ ]:
#Importar base atualizada de parcelas como table
#APONTAR O CAMINHO DA BASE DO GEO
arcpy.TableToDBASE_conversion(r'C:\Users\KLDB00132941\Documents\Klabin\IFS_VANT\IFS_VANT_TEMP.gdb\IFS_PARCELA_DEV',gdb)


In [ ]:
# Criar coluna id_talhao_2 para abrigar código talhão com apenas 6 ítens:

#Set variables
inFeatures = local + "\\" + projname + "\\" + projname + ".gdb" + "\\" + "IFS_PARCELA_DEV"
fieldName = "id_tlh_2"

arcpy.AddField_management(inFeatures,fieldName,"TEXT",6 )

In [ ]:
arcpy.management.CalculateField(inFeatures,"id_tlh_2", "!ID_TALHAO![4:]", "PYTHON3", None)


In [ ]:
#Juntar bases para definir o que ainda não foi medido:
pos_atual = local + "\\" + projname + "\\" + projname + ".gdb" + "\\" + "pos_atualizado"

joinpos = arcpy.management.AddJoin(pos_atual, "Talhão_P", inFeatures, "id_tlh_2", "KEEP_ALL")

In [ ]:
#Selecionar somente o que não foi realizado e status de "MODIFICADO":
arcpy.management.SelectLayerByAttribute(joinpos, "NEW_SELECTION", "IFS_PARCELA_DEV.id_tlh_2 IS NULL And pos_atualizado.Operação = 'MODIFICADO'", None)

In [ ]:
# Salvar somente linhas SEM match:
talhao_sem_match = local + "\\" + projname + "\\" + projname + ".gdb" + "\\" + "talhao_sem_match"
arcpy.management.CopyRows(joinpos, talhao_sem_match, None)

In [ ]:
#Delete fields desnecessários
arcpy.DeleteField_management(talhao_sem_match,"IFS_PARCELA_id_tlh_2;IFS_PARCELA_Shape_Length;IFS_PARCELA_last_edited_user;IFS_PARCELA_last_edited_date;IFS_PARCELA_created_user;IFS_PARCELA_created_date;IFS_PARCELA_Shape__Length;IFS_PARCELA_Shape__Length;IFS_PARCELA_OBJECTID;IFS_PARCELA_ID_TALHAO;IFS_PARCELA_OBS;IFS_PARCELA_ID_PARCELA;IFS_PARCELA_DATA_PLANTIO;IFS_PARCELA_SUBTIPO;IFS_PARCELA_DATA_VOO;IFS_PARCELA_IMG;IFS_PARCELA_Shape__Area")

In [ ]:
# Add field com id_talhao
arcpy.management.AddField(r"C:\Users\KLDB00132941\Documents\Klabin\IFS_VANT\DEV\Cadastro\Cadastro.gdb\Cadastro_17_19_select", "id_talhao", "TEXT", None, None, 6, None, "NULLABLE", "NON_REQUIRED", None)

In [ ]:
arcpy.management.CalculateField(r"C:\Users\KLDB00132941\Documents\Klabin\IFS_VANT\DEV\Cadastro\Cadastro.gdb\Cadastro_17_19_select", "id_talhao", "!Objeto_de_locação!", "PYTHON3", None)

In [ ]:
#Join com Cadastro
talhao_join = arcpy.management.AddJoin(talhao_sem_match, "pos_atualizado_Talhão_P", r"C:\Users\KLDB00132941\Documents\Klabin\IFS_VANT\DEV\Cadastro\Cadastro.gdb\Cadastro_17_19_select", "Objeto_de_Locação", "KEEP_COMMON")


In [ ]:
#Lista final de talhoes a ser analisados:

talhao_cad = local + "\\" + projname + "\\" + projname + ".gdb" + "\\" + "talhao_cad"
arcpy.management.CopyRows(talhao_join, talhao_cad, None)

Calcular número de parcelas

In [ ]:
#Criar field 
arcpy.management.AddField(talhao_cad, "n_parcelas", "SHORT", None, None, None, None, "NULLABLE", "NON_REQUIRED", None)

In [ ]:
#Cálculo do número de parcelas arredonado (=Round (AP/2)):
arcpy.management.CalculateField(talhao_cad, "n_parcelas", "!Cadastro_17_19_select_Área_Produtiva__ha_!/2", "PYTHON3", None)

In [ ]:
# Add field com id_talhao_2 (Centro+RF+TALHAO)
arcpy.management.AddField(talhao_cad, "id_talhao_2", "TEXT", None, None, 12, None, "NULLABLE", "NON_REQUIRED", None)

In [ ]:
#Calculando o id_talhao_2
arcpy.management.CalculateField(talhao_cad, "id_talhao_2", "!Cadastro_17_19_select_Centro! + !Cadastro_17_19_select_id_talhao!", "PYTHON3", None)

In [ ]:
#Copiando talhões de interesse

In [ ]:
#Add join Área produtiva e talhoes_cad
#Área produtiva seria a base do GEO oficial
APJOIN = arcpy.management.AddJoin(r"C:\Users\KLDB00132941\Documents\Klabin\IFS_VANT\DEV\ifs_vant_talhoes.gdb\AREA_PRODUTIVA\AREA_PRODUTIVA", "ID_TALHAO", talhao_cad, "id_talhao_2", "KEEP_COMMON")

In [ ]:
arcpy.conversion.FeatureClassToFeatureClass(APJOIN, local + "\\" + projname + "\\" + projname + ".gdb", "AP"+ Date, None, 'TIPO "TIPO" true true false 2 Short 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.TIPO,-1,-1;ID_TALHAO "ID_TALHAO" true true false 10 Text 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.ID_TALHAO,0,10;DATA_PLANTIO "DATA_PLANTIO" true true false 8 Date 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.DATA_PLANTIO,-1,-1;PercKlb "PercKlb" true true false 8 Double 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.PercKlb,-1,-1;PROPRIEDADE "PROPRIEDADE" true true false 4 Text 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.PROPRIEDADE,0,4;STATUS "STATUS" true true false 5 Text 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.STATUS,0,5;SUBTIPO "SUBTIPO" true true false 4 Text 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.SUBTIPO,0,4;created_user "created_user" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.created_user,0,255;created_date "created_date" true true false 8 Date 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.created_date,-1,-1;last_edited_user "last_edited_user" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.last_edited_user,0,255;last_edited_date "last_edited_date" true true false 8 Date 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.last_edited_date,-1,-1;GlobalID "GlobalID" false false false 38 GlobalID 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.GlobalID,-1,-1;STATUS_NEGOCIACAO "STATUS_NEGOCIACAO" true true false 2 Short 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.STATUS_NEGOCIACAO,-1,-1;PROJETO "PROJETO" true true false 50 Text 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.PROJETO,0,50;BOOK "BOOK" true true false 5 Text 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.BOOK,0,5;RECOMEN_ESP "RECOMEN_ESP" true true false 3 Text 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.RECOMEN_ESP,0,3;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.Shape_Area,-1,-1;id_talhao_s_centro "id_talhao_s_centro" true true false 20 Text 0 0,First,#,AREA_PRODUTIVA_Layer,AREA_PRODUTIVA.id_talhao_s_centro,0,20;OBJECTID "OBJECTID" false true false 4 Long 0 9,First,#,AREA_PRODUTIVA_Layer,talhao_cad.OBJECTID,-1,-1;talhao_sem_match_pos_atualizado_ID_2 "ID_2" true true false 254 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_pos_atualizado_ID_2,0,254;talhao_sem_match_pos_atualizado_Talhão_Pr "Talhão_Pr" true true false 254 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_pos_atualizado_Talhão_Pr,0,254;talhao_sem_match_pos_atualizado_Talhão_P "Talhão_P" true true false 254 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_pos_atualizado_Talhão_P,0,254;talhao_sem_match_pos_atualizado_Área_Pré "Área_Pré" true true false 8 Double 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_pos_atualizado_Área_Pré,-1,-1;talhao_sem_match_pos_atualizado_Área_Pós "Área_Pós" true true false 8 Double 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_pos_atualizado_Área_Pós,-1,-1;talhao_sem_match_pos_atualizado_Operação "Operação" true true false 254 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_pos_atualizado_Operação,0,254;talhao_sem_match_pos_atualizado_Edição_T "Edição_T" true true false 254 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_pos_atualizado_Edição_T,0,254;talhao_sem_match_pos_atualizado_Criado "Criado" true true false 8 Date 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_pos_atualizado_Criado,-1,-1;talhao_sem_match_pos_atualizado_Editado_po "Editado_po" true true false 254 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_pos_atualizado_Editado_po,0,254;talhao_sem_match_pos_atualizado_Modificado "Modificado" true true false 254 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_pos_atualizado_Modificado,0,254;talhao_sem_match_pos_atualizado_Criado_Esp "Criado_Esp" true true false 8 Date 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_pos_atualizado_Criado_Esp,-1,-1;talhao_sem_match_pos_atualizado_Recebeu_á "Recebeu_á" true true false 254 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_pos_atualizado_Recebeu_á,0,254;talhao_sem_match_pos_atualizado_Tipo_de_It "Tipo_de_It" true true false 254 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_pos_atualizado_Tipo_de_It,0,254;talhao_sem_match_pos_atualizado_Caminho "Caminho" true true false 254 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_pos_atualizado_Caminho,0,254;talhao_sem_match_IFS_PARCELA_Shape_Area "Shape_Area" true true false 8 Double 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.talhao_sem_match_IFS_PARCELA_Shape_Area,-1,-1;Cadastro_18_19_select_OBJECTID "OBJECTID" true true false 4 Long 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_OBJECTID,-1,-1;Cadastro_18_19_select_Tipo_utiliz_da_UL "Tipo utiliz.da UL" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Tipo_utiliz_da_UL,0,255;Cadastro_18_19_select_Terreno "Terreno" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Terreno,0,255;Cadastro_18_19_select_Objeto_de_locação "Objeto de locação" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Objeto_de_locação,0,255;Cadastro_18_19_select_Denom_objeto_locação "Denom.objeto locação" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Denom_objeto_locação,0,255;Cadastro_18_19_select_Status "Status" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Status,0,255;Cadastro_18_19_select_Status_1 "Status" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Status_1,0,255;Cadastro_18_19_select_Município "Município" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Município,0,255;Cadastro_18_19_select_CEP "CEP" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_CEP,0,255;Cadastro_18_19_select_Área_Produtiva__ha_ "Área Produtiva (ha)" true true false 8 Double 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Área_Produtiva__ha_,-1,-1;Cadastro_18_19_select_Área_Plantada__ha_ "Área Plantada (ha)" true true false 8 Double 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Área_Plantada__ha_,-1,-1;Cadastro_18_19_select_Área_colhida__ha_ "Área colhida (ha)" true true false 8 Double 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Área_colhida__ha_,-1,-1;Cadastro_18_19_select_Área_Plantada_Falha_Resíduo_HA "Área Plantada Falha/Resíduo HA" true true false 8 Double 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Área_Plantada_Falha_Resíduo_HA,-1,-1;Cadastro_18_19_select_Área_sem_plantio__ha_ "Área sem plantio (ha)" true true false 8 Double 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Área_sem_plantio__ha_,-1,-1;Cadastro_18_19_select_Área_APPF "Área APPF" true true false 8 Double 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Área_APPF,-1,-1;Cadastro_18_19_select_SCP "SCP" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_SCP,0,255;Cadastro_18_19_select_Tipo_Talhão "Tipo Talhão" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Tipo_Talhão,0,255;Cadastro_18_19_select_Propriedade_da_Floresta "Propriedade da Floresta" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Propriedade_da_Floresta,0,255;Cadastro_18_19_select_Tipo_de_área "Tipo de área" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Tipo_de_área,0,255;Cadastro_18_19_select_Vegetação_Original "Vegetação Original" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Vegetação_Original,0,255;Cadastro_18_19_select_Tipo_de_Solo "Tipo de Solo" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Tipo_de_Solo,0,255;Cadastro_18_19_select_Textura_do_Solo "Textura do Solo" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Textura_do_Solo,0,255;Cadastro_18_19_select_Ordem_Interna "Ordem Interna" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Ordem_Interna,0,255;Cadastro_18_19_select_Bloco_colheita "Bloco colheita" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Bloco_colheita,0,255;Cadastro_18_19_select_Região_Colheita "Região Colheita" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Região_Colheita,0,255;Cadastro_18_19_select_Logabilidade "Logabilidade" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Logabilidade,0,255;Cadastro_18_19_select_Planejamento_Curto_Prazo "Planejamento Curto Prazo" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Planejamento_Curto_Prazo,0,255;Cadastro_18_19_select_Planejamento_médio_prazo "Planejamento médio prazo" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Planejamento_médio_prazo,0,255;Cadastro_18_19_select_Espécie_Rec_Técnica "Espécie Rec.Técnica" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Espécie_Rec_Técnica,0,255;Cadastro_18_19_select_Classe_do_Sítio "Classe do Sítio" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Classe_do_Sítio,0,255;Cadastro_18_19_select_Potencial_de_Produtividade_Pinus "Potencial de Produtividade Pinus" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Potencial_de_Produtividade_Pinus,0,255;Cadastro_18_19_select_Potencial_de_Produtividade_Eucalipto "Potencial de Produtividade Eucalipto" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Potencial_de_Produtividade_Eucalipto,0,255;Cadastro_18_19_select_Unidade_de_manejo "Unidade de manejo" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Unidade_de_manejo,0,255;Cadastro_18_19_select_Ciclo_Plantio "Ciclo Plantio" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Ciclo_Plantio,0,255;Cadastro_18_19_select_Data_Inicio_Ciclo "Data Inicio Ciclo" true true false 8 Date 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Data_Inicio_Ciclo,-1,-1;Cadastro_18_19_select_Data_Fim_Ciclo "Data Fim Ciclo" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Data_Fim_Ciclo,0,255;Cadastro_18_19_select_Data_de_Plantio "Data de Plantio" true true false 8 Date 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Data_de_Plantio,-1,-1;Cadastro_18_19_select_Condição "Condição" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Condição,0,255;Cadastro_18_19_select_Fase "Fase" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Fase,0,255;Cadastro_18_19_select_Tipo_de_Plantio "Tipo de Plantio" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Tipo_de_Plantio,0,255;Cadastro_18_19_select_Tipo_de_Manejo "Tipo de Manejo" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Tipo_de_Manejo,0,255;Cadastro_18_19_select_Tipo_Experimento "Tipo Experimento" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Tipo_Experimento,0,255;Cadastro_18_19_select_Quantidade_de_mudas_plantadas "Quantidade de mudas plantadas" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Quantidade_de_mudas_plantadas,0,255;Cadastro_18_19_select_Certificação_FSC "Certificação FSC" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Certificação_FSC,0,255;Cadastro_18_19_select_Espaçamento_entre_Linhas "Espaçamento entre Linhas" true true false 8 Double 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Espaçamento_entre_Linhas,-1,-1;Cadastro_18_19_select_Espaçamento_entre_Plantas "Espaçamento entre Plantas" true true false 8 Double 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Espaçamento_entre_Plantas,-1,-1;Cadastro_18_19_select_Projeto_Silvicultura "Projeto Silvicultura" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Projeto_Silvicultura,0,255;Cadastro_18_19_select_Rotação "Rotação" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Rotação,0,255;Cadastro_18_19_select_Data_de_Inicio "Data de Inicio" true true false 8 Date 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Data_de_Inicio,-1,-1;Cadastro_18_19_select_Data_fim_da_rotação "Data fim da rotação" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Data_fim_da_rotação,0,255;Cadastro_18_19_select_Ultima_intervenção "Ultima intervenção" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Ultima_intervenção,0,255;Cadastro_18_19_select_Data_da_ultima_intervenção "Data da ultima intervenção" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Data_da_ultima_intervenção,0,255;Cadastro_18_19_select_Linha_de_desbaste_sistemático "Linha de desbaste sistemático" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Linha_de_desbaste_sistemático,0,255;Cadastro_18_19_select_Desbaste_Seletivo "Desbaste Seletivo" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Desbaste_Seletivo,0,255;Cadastro_18_19_select_Última_desrama "Última desrama" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Última_desrama,0,255;Cadastro_18_19_select_Data_Última_desrama "Data Última desrama" true true false 8 Date 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Data_Última_desrama,-1,-1;Cadastro_18_19_select_Altura_de_desrama "Altura de desrama" true true false 4 Long 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Altura_de_desrama,-1,-1;Cadastro_18_19_select_Espécies "Espécies" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Espécies,0,255;Cadastro_18_19_select_Descrição_Espécie "Descrição Espécie" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Descrição_Espécie,0,255;Cadastro_18_19_select_Procedência_do_material_genético "Procedência do material genético" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Procedência_do_material_genético,0,255;Cadastro_18_19_select_Origem_Material_Genético "Origem Material Genético" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Origem_Material_Genético,0,255;Cadastro_18_19_select_Clone_Semente "Clone Semente" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Clone_Semente,0,255;Cadastro_18_19_select_Viveiro "Viveiro" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Viveiro,0,255;Cadastro_18_19_select_Local_Coleta "Local Coleta" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Local_Coleta,0,255;Cadastro_18_19_select_Geração "Geração" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Geração,0,255;Cadastro_18_19_select_Família "Família" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Família,0,255;Cadastro_18_19_select_Propagação "Propagação" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Propagação,0,255;Cadastro_18_19_select_Número_do_Contrato "Número do Contrato" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Número_do_Contrato,0,255;Cadastro_18_19_select_Denom_tipo_contrato "Denom.tipo contrato" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Denom_tipo_contrato,0,255;Cadastro_18_19_select_Proprietario "Proprietario" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Proprietario,0,255;Cadastro_18_19_select_F__Total_Proprietario "% Total Proprietario" true true false 4 Long 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_F__Total_Proprietario,-1,-1;Cadastro_18_19_select_Início_do_contrato "Início do contrato" true true false 8 Date 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Início_do_contrato,-1,-1;Cadastro_18_19_select_Data_Fim_de_Contrato "Data Fim de Contrato" true true false 8 Date 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Data_Fim_de_Contrato,-1,-1;Cadastro_18_19_select_Data_Fim_com_Carência "Data Fim com Carência" true true false 8 Date 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Data_Fim_com_Carência,-1,-1;Cadastro_18_19_select_Centro "Centro" true true false 255 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_Centro,0,255;Cadastro_18_19_select_id_talhao "id_talhao" true true false 6 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.Cadastro_18_19_select_id_talhao,0,6;n_parcelas "n_parcelas" true true false 2 Short 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.n_parcelas,-1,-1;id_talhao_2 "id_talhao_2" true true false 12 Text 0 0,First,#,AREA_PRODUTIVA_Layer,talhao_cad.id_talhao_2,0,12', None)

In [ ]:
#Buffer negativo de 11 metros para que parcelas não sejam plotadas fora dos talhões
#Também dissolve por ID_TALHAO  e SUBTIPO
arcpy.analysis.Buffer(local + "\\" + projname + "\\" + projname + ".gdb" + "\\" + "AP"+ Date, r"C:\Users\KLDB00132941\Documents\Klabin\IFS_VANT\DEV" + "\\" + projname + "\\" + projname + ".gdb" + "\\" + "Buf_AP"+ Date, "-11 Meters", "FULL", "ROUND", "LIST", "ID_TALHAO;SUBTIPO", "PLANAR")

Criação de parcelas

In [ ]:
# Join com talhao_cad para obter coluna com número de parcelas
APJOIN = arcpy.management.AddJoin(local + "\\" + projname + "\\" + projname + ".gdb" + "\\" + "Buf_AP"+ Date, "ID_TALHAO", talhao_cad, "id_talhao_2", "KEEP_COMMON")

In [ ]:
#Plotagem das percelas
arcpy.management.CreateRandomPoints(local + "\\" + projname + "\\" + projname + ".gdb", "point_parcelas", APJOIN, "0 0 250 250", "talhao_cad.n_parcelas", "22 Meters", "POINT", 0)

Calcular diferença no número de parcelas e DECIDIR se será replotado automáticamente ou será liberado relatório para plotagem manual

In [ ]:
#Spatial join com o footprint das imagens
#SUBSTITUIR O FOOTPRINT DA BASE DO GEO
sp_parcelas = local + "\\" + projname + "\\" + projname + ".gdb" + "\\" + "spatialjoin_parcelas"
arcpy.analysis.SpatialJoin(local + "\\" + projname + "\\" + projname + ".gdb" + "\\" + "point_parcelas", r'C:\Users\KLDB00132941\Documents\Klabin\IFS_VANT\DEV\New File Geodatabase.gdb' + "\\" + 'footprint_3'  , sp_parcelas, "JOIN_ONE_TO_ONE", "KEEP_ALL", 'CID "CID" true true false 4 Long 0 0,First,#,point_parcelas,CID,-1,-1;Raster "Raster" true true false 0 Raster 0 0,First,#,footprint_3,Raster,-1,-1;Name "Name" true true false 200 Text 0 0,First,#,footprint_3,Name,0,200;MinPS "MinPS" true true false 8 Double 0 0,First,#,footprint_3,MinPS,-1,-1;MaxPS "MaxPS" true true false 8 Double 0 0,First,#,footprint_3,MaxPS,-1,-1;LowPS "LowPS" true true false 8 Double 0 0,First,#,footprint_3,LowPS,-1,-1;HighPS "HighPS" true true false 8 Double 0 0,First,#,footprint_3,HighPS,-1,-1;Category "Category" true true false 4 Long 0 0,First,#,footprint_3,Category,-1,-1;Tag "Tag" true true false 100 Text 0 0,First,#,footprint_3,Tag,0,100;GroupName "GroupName" true true false 100 Text 0 0,First,#,footprint_3,GroupName,0,100;ProductName "ProductName" true true false 100 Text 0 0,First,#,footprint_3,ProductName,0,100;CenterX "CenterX" true true false 8 Double 0 0,First,#,footprint_3,CenterX,-1,-1;CenterY "CenterY" true true false 8 Double 0 0,First,#,footprint_3,CenterY,-1,-1;ZOrder "ZOrder" true true false 4 Long 0 0,First,#,footprint_3,ZOrder,-1,-1;TypeID "Raster Type ID" true true false 4 Long 0 0,First,#,footprint_3,TypeID,-1,-1;ItemTS "ItemTS" true true false 8 Double 0 0,First,#,footprint_3,ItemTS,-1,-1;UriHash "UriHash" true true false 50 Text 0 0,First,#,footprint_3,UriHash,0,50;Uri "Uri" true true false 0 Blob 0 0,First,#,footprint_3,Uri,-1,-1;DATA "DATA" true true false 8 Date 0 0,First,#,footprint_3,DATA,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,footprint_3,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,footprint_3,Shape_Area,-1,-1', "INTERSECT", None, None)